# Example - parallel computation

In [2]:
import numpy as np
import pygmsh
from ttcrpy import tmesh
from numpy.random import default_rng

Define a function for slowness

In [18]:
def f(z):
    return 1.0 / (1.5 + 4.5*z)

In [19]:
geom = pygmsh.built_in.Geometry()
lcar = 0.042
geom.add_box(0.0, 1.0, 0.0, 1.0, 0.0, 1.0, lcar=lcar)
msh = pygmsh.generate_mesh(geom)

Info    : Running 'gmsh -3 /var/folders/j6/s0152dxj3q54bsjvw2s42b0m0000gn/T/tmpkcevih03.geo -format msh -bin -o /var/folders/j6/s0152dxj3q54bsjvw2s42b0m0000gn/T/tmpf2quna1_.msh' [Gmsh 4.5.6, 1 node, max. 1 thread]
Info    : Started on Sat Aug 22 21:43:46 2020
Info    : Reading '/var/folders/j6/s0152dxj3q54bsjvw2s42b0m0000gn/T/tmpkcevih03.geo'...
Info    : Done reading '/var/folders/j6/s0152dxj3q54bsjvw2s42b0m0000gn/T/tmpkcevih03.geo'
Info    : Meshing 1D...
Info    : [  0 %] Meshing curve 1 (Line)
Info    : [ 10 %] Meshing curve 2 (Line)
Info    : [ 20 %] Meshing curve 3 (Line)
Info    : [ 30 %] Meshing curve 4 (Line)
Info    : [ 40 %] Meshing curve 5 (Line)
Info    : [ 50 %] Meshing curve 6 (Line)
Info    : [ 50 %] Meshing curve 7 (Line)
Info    : [ 60 %] Meshing curve 8 (Line)
Info    : [ 70 %] Meshing curve 9 (Line)
Info    : [ 80 %] Meshing curve 10 (Line)
Info    : [ 90 %] Meshing curve 11 (Line)
Info    : [100 %] Meshing curve 12 (Line)
Info    : Done meshing 1D (0.001092 s)
Info

Create some variables

In [20]:
slowness = f(msh.points[:,2])
n_threads = np.arange(2, 9)

rng = default_rng(1966)

src = rng.uniform(0.01, 0.99, (100, 3))
rcv = rng.uniform(0.05, 0.95, (100, 3))


In [21]:
tm = tmesh.Mesh3d(msh.points, tetra=msh.cells_dict['tetra'], method='DSPM', cell_slowness=0)
tm.set_slowness(slowness)

In [22]:
tt = tm.raytrace(src[:10,:], rcv)

AttributeError: 'ttcrpy.tmesh.Mesh3d' object has no attribute 'is_outside'

In [4]:
pts = np.arange(12)
pts.reshape((4,3))
for p in pts:
    print(p)

0
1
2
3
4
5
6
7
8
9
10
11
